In [43]:
from sklearn.base import ClassifierMixin
import pandas as pd
import numpy as np



In [58]:
class logreg(ClassifierMixin):
    def __init__(self, lr = 0.01, epochs = 200):
        super().__init__()
        self.lr = lr
        self.epochs = epochs
        self.w = None
        self.b = None

    def sigmoid(self, z):
        return 1 / (1 + np.e**(-z))
    

    def fit(self, X:np.ndarray, Y:np.ndarray, pos_class):
        n, d = X.shape
        self.w = np.random.random(d)
        self.b = np.random.randint(0,5) 
        Y = (Y == pos_class).astype(int)
        for _ in range(self.epochs):
            idx = np.random.permutation(n)
            for i in idx: 
                er = Y[i] - self.sigmoid(np.dot(self.w, X[i]) + self.b)
                w_grad = - X[i] * er
                b_grad = - er
                self.w -= self.lr * w_grad
                self.b -= self.lr * b_grad
        return self
    
    def predict(self, X:np.ndarray):
        n,d = X.shape
        ans = []
        for i in range(n):
            p = self.sigmoid(np.dot(X[i], self.w) + self.b)
            ans.append(p)
        return np.array(ans)

            

In [45]:
from sklearn.datasets import load_iris
X, Y = load_iris()['data'], load_iris()['target']

In [46]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

# lr = logreg()
# lr.fit(X_train, y_train)
# pred = lr.predict(X_test)
# mean_absolute_error(pred, y_test)


У нас получилось что-то работающее. Обобщим для многоклассовой классификации

In [47]:
import numpy as np

In [48]:
a = [1,2,3]
exponented = np.exp(a)
exponented

array([ 2.71828183,  7.3890561 , 20.08553692])

In [ ]:

class smax(ClassifierMixin):
    def __init__(self, lr = 0.01, epochs = 200):
        super().__init__()
        self.lr = lr
        self.epochs = epochs

        self.logregs = {}


    def fit(self, X:np.ndarray, Y:np.ndarray):
        classes = set(Y)
        self.logregs = {}
        for cl in classes:
            lg = logreg()
            lg.fit(X,Y,cl)
            self.logregs[cl] = lg
        return self
            
    def softmax(self, a):
        exponented = np.exp(a)
        exp_sum = np.sum(exponented)
        return exponented/exp_sum

    def predict(self, X:np.ndarray, probas = False):
        n, d = X.shape
        ans = []
        for i in range(n):
            y = []
            y_classes = []
            for cl, lg in self.logregs.items():
                y.append(float(lg.predict(X[i].reshape(1,-1))))
                y_classes.append(int(cl))
            y = [float(i) for i in list(self.softmax(y))]
            res = dict(zip(y_classes, y))
            ans.append(res)
        if probas:
            return ans
        return [max(i, key=i.get) for i in ans]
            


In [75]:
from sklearn.metrics import accuracy_score
sm = smax()

sm.fit(X_train,y_train)
pred = sm.predict(X_test,probas=True)
# accuracy_score(pred, y_test)
pred

C:\Users\Maxim\AppData\Local\Temp\ipykernel_4648\4096874229.py:31: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  y.append(float(lg.predict(X[i].reshape(1,-1))))


[{0: 0.2619647972004547, 1: 0.4552178014731016, 2: 0.28281740132644373},
 {0: 0.5677491105487567, 1: 0.22220213733612973, 2: 0.21004875211511356},
 {0: 0.175856356877825, 1: 0.3465290104931324, 2: 0.4776146326290425},
 {0: 0.28920380969325715, 1: 0.3906369301832525, 2: 0.32015926012349044},
 {0: 0.26695658029323915, 1: 0.4572965947710886, 2: 0.27574682493567226},
 {0: 0.5631801043804077, 1: 0.22816950984156856, 2: 0.2086503857780237},
 {0: 0.3111123050665055, 1: 0.3813093466643938, 2: 0.3075783482691007},
 {0: 0.2613909756629575, 1: 0.28975527633837644, 2: 0.44885374799866595},
 {0: 0.22090276046055507, 1: 0.4539840362730244, 2: 0.32511320326642057},
 {0: 0.28265924604745923, 1: 0.4334259632312212, 2: 0.2839147907213196},
 {0: 0.26859028714549865, 1: 0.30425163786015846, 2: 0.42715807499434294},
 {0: 0.5402178306599166, 1: 0.2597440714964907, 2: 0.20003809784359278},
 {0: 0.5637831684326371, 1: 0.22835243656335164, 2: 0.20786439500401122},
 {0: 0.5433150652866329, 1: 0.2552964765827112

Неплохо